<a href="https://colab.research.google.com/github/kamlallu/EDA/blob/main/RAG_flask_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers faiss-cpu sentence-transformers PyPDF2

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
pip install PyMuPDF

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
pip install -U langchain-community


[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import fitz

model_name = "google/flan-t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
text = ""

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_file_path):
    text = ""
    with fitz.open(pdf_file_path) as pdf_document:
        num_pages = pdf_document.page_count
        for page_num in range(num_pages):
            page = pdf_document[page_num]
            text += page.get_text()
    return text

# Path to the PDF file
pdf_file_path = "C:/Users/priya_0yiyr9i/Downloads/Meta-03-31-2024-Exhibit-99-1_FINAL.pdf"

# Extract text from the PDF
pdf_text = extract_text_from_pdf(pdf_file_path)

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Initialize T5 model and tokenizer
model_name = "google/flan-t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Dummy function to retrieve relevant passages (replace with actual retrieval logic)
def retrieve_relevant_passages(question):
    # Placeholder logic (replace with actual retrieval logic)
    return "Meta Platforms, Inc. is a global technology company that develops platforms for people to connect."

# Define multiple questions
questions = [
    "What is the main topic of the document?",
    "Who are the key stakeholders mentioned?",
    "What are the financial highlights discussed?",
    "What is the conclusion of the report?",
]

# Iterate through questions and generate answers using RAG
for idx, question in enumerate(questions, start=1):
    print(f"Question {idx}: {question}")

    # Retrieve relevant passages
    retrieved_passages = retrieve_relevant_passages(question)

    # Generate answer using RAG approach
    inputs = tokenizer.encode("question: " + question + " context: " + retrieved_passages, return_tensors="pt", max_length=512, truncation=True)

    # Adjust max_length for longer answers
    outputs = model.generate(inputs, max_length=1000)  # Increase max_length as needed

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print(f"Answer {idx}: {answer}")
    print()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Question 1: What is the main topic of the document?
Answer 1: Meta Platforms

Question 2: Who are the key stakeholders mentioned?
Answer 2: Meta Platforms

Question 3: What are the financial highlights discussed?
Answer 3: Meta Platforms, Inc. is a global technology company that develops platforms for people to connect.

Question 4: What is the conclusion of the report?
Answer 4: It is a global technology company that develops platforms for people to connect.



In [ ]:
pip install dash

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: dash in c:\users\priya_0yiyr9i\anaconda3\lib\site-packages (2.17.1)




[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from transformers import T5ForConditionalGeneration, T5Tokenizer
import socket

# Initialize Dash app
app = dash.Dash(__name__)

# Initialize T5 model and tokenizer
model_name = "google/flan-t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Dummy function to retrieve relevant passages
def retrieve_relevant_passages(question):
    return "Meta Platforms, Inc. is a global technology company that develops platforms for people to connect."

# Define app layout
app.layout = html.Div([
    html.H1("Q&A System"),

    # Input field for question
    dcc.Input(id='question-input', type='text', placeholder='Enter your question', style={'width': '60%'}),

    # Button to submit question
    html.Button('Get Answer', id='submit-button', n_clicks=0),

    # Output area for answer
    html.Div(id='answer-output', style={'marginTop': '20px'}),

    # Display the server link
    html.Div(id='server-link-output', style={'marginTop': '20px'})
])

# Define callback to update the answer and server link
@app.callback(
    [Output('answer-output', 'children'),
     Output('server-link-output', 'children')],
    [Input('submit-button', 'n_clicks')],
    [dash.dependencies.State('question-input', 'value')]
)
def update_answer(n_clicks, question):
    if n_clicks > 0:
        if question:
            # Retrieve relevant passages
            retrieved_passages = retrieve_relevant_passages(question)

            # Generate answer
            inputs = tokenizer.encode("question: " + question + " context: " + retrieved_passages, return_tensors="pt", max_length=512, truncation=True)
            outputs = model.generate(inputs, max_length=1000)
            answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

            # Display the server link
            server_link = f"http://127.0.0.1:8050"  # Default local server address for Dash

            return f"Answer: {answer}", f"Server Link: {server_link}"
        else:
            return "Please enter a question.", ""
    return "", ""

if __name__ == '__main__':
    app.run_server(debug=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
